In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from qiskit import QuantumCircuit, Aer, execute
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load the dataset
df = pd.read_csv("C:\\Users\\riakh\\Downloads\\Earthquake Prediction Using QML\\earthquaketrain.csv\\train.csv")

# Extract features and target
X = df['acoustic_data'].values.reshape(-1, 1)
y = df['time_to_failure'].values

# Scale the data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)  # Validation set

# Restrict the data to 2000 points for training
train_size = 2000
X_train_subset = X_train[:train_size]
y_train_subset = y_train[:train_size]

# Quantum Feature Map with Hadamard Gates
def quantum_feature_map(data, num_qubits=4):
    """Creates a quantum circuit for the feature map with Hadamard gates."""
    qc = QuantumCircuit(num_qubits)
    
    # Apply Hadamard gate to all qubits initially
    for i in range(num_qubits):
        qc.h(i)
    
    # Apply Ry rotation based on the input data
    for i, value in enumerate(data):
        qc.ry(value * np.pi, i % num_qubits)  # Use Ry rotation to encode data
    return qc

# Quantum Ansatz
def quantum_ansatz(num_qubits=4, depth=2, parameters=None):
    qc = QuantumCircuit(num_qubits)
    if parameters is None:
        parameters = np.random.rand(num_qubits * depth * 2)  # Create random parameters if not provided
    
    param_idx = 0
    for d in range(depth):
        for i in range(num_qubits):
            qc.rx(parameters[param_idx], i)
            param_idx += 1
            qc.ry(parameters[param_idx], i)
            param_idx += 1
        
        for i in range(num_qubits - 1):
            qc.cx(i, i + 1)  # Add entanglement using CNOT gates
    
    return qc, parameters

# Measurement and Execution
def measure_and_execute(qc):
    qc.measure_all()
    simulator = Aer.get_backend('aer_simulator')
    result = execute(qc, backend=simulator, shots=1024).result()
    counts = result.get_counts(qc)
    
    output_value = sum([int(key, 2) * value for key, value in counts.items()]) / 1024
    return output_value

# Hybrid Quantum-Classical Model with Neural Network
class HybridQuantumModel:
    def __init__(self, num_qubits=4, depth=2):
        self.num_qubits = num_qubits
        self.depth = depth
        self.parameters = np.random.rand(num_qubits * depth * 2)
        self.build_classical_model()
        self.train_losses = []  # List to store training loss
        self.val_losses = []  # List to store validation loss

    def build_classical_model(self):
        # Define a more complex classical model
        self.classical_model = Sequential([
            Dense(64, activation='relu', input_shape=(1,)),
            Dropout(0.3),
            Dense(32, activation='relu'),
            Dropout(0.3),
            Dense(16, activation='relu'),
            Dense(1, activation='linear')
        ])
        self.classical_model.compile(optimizer='adam', loss='mse')
    
    def quantum_forward(self, x):
        qc = quantum_feature_map(x, num_qubits=self.num_qubits)
        ansatz, _ = quantum_ansatz(self.num_qubits, self.depth, self.parameters)
        qc.compose(ansatz, inplace=True)
        
        quantum_output = measure_and_execute(qc)
        return quantum_output
    
    def fit(self, X_train, y_train, X_val, y_val, epochs=100):
        for epoch in range(epochs):  # Number of training epochs
            epoch_loss = 0
            quantum_outputs_train = []
            quantum_outputs_val = []
            
            # Forward pass for training data
            for i in range(len(X_train)):
                quantum_output = self.quantum_forward(X_train[i])
                quantum_outputs_train.append([quantum_output])
                
            quantum_outputs_train = np.array(quantum_outputs_train)
            history = self.classical_model.fit(quantum_outputs_train, y_train, epochs=1, verbose=0)
            avg_epoch_loss = history.history['loss'][0]
            self.train_losses.append(avg_epoch_loss)
            
            # Forward pass for validation data
            for i in range(len(X_val)):
                quantum_output_val = self.quantum_forward(X_val[i])
                quantum_outputs_val.append([quantum_output_val])
            
            quantum_outputs_val = np.array(quantum_outputs_val)
            val_loss = self.classical_model.evaluate(quantum_outputs_val, y_val, verbose=0)
            self.val_losses.append(val_loss)

            print(f'Epoch {epoch + 1}/{epochs}, Loss: {avg_epoch_loss:.6f}, Val Loss: {val_loss:.6f}')

    def predict(self, X):
        quantum_outputs = []
        for i in range(len(X)):
            quantum_output = self.quantum_forward(X[i])
            quantum_outputs.append([quantum_output])
        
        quantum_outputs = np.array(quantum_outputs)
        predictions = self.classical_model.predict(quantum_outputs)
        return predictions.flatten()

# Initialize and train the hybrid model with the subset of data
hybrid_model = HybridQuantumModel(num_qubits=4, depth=2)
hybrid_model.fit(X_train_subset, y_train_subset, X_val, y_val, epochs=100)

# Plot the training and validation loss curves
plt.plot(hybrid_model.train_losses, label='Training Loss')
plt.plot(hybrid_model.val_losses, label='Validation Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Predict on test data
predictions = hybrid_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error on Test Data: {mse}')


Epoch 1/100, Loss: 0.594863, Val Loss: 0.116077
Epoch 2/100, Loss: 0.172141, Val Loss: 0.187015
Epoch 3/100, Loss: 0.112132, Val Loss: 0.180309
Epoch 4/100, Loss: 0.080820, Val Loss: 0.175982
Epoch 5/100, Loss: 0.069453, Val Loss: 0.168947


KeyboardInterrupt: 